## Lesson 2: LangGraph Components

https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/3/langgraph-components

In [2]:
import operator
from typing import Annotated, TypedDict

from dotenv import find_dotenv, load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph

In [5]:
load_dotenv(find_dotenv())

MODEL = "glm-4"
BASE_URL = "https://open.bigmodel.cn/api/paas/v4/"

In [6]:
tool = TavilySearchResults(max_results=4)

print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [7]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [8]:
class Agent:
    def __init__(self, model, tools, system: str = "") -> None:
        self.system = system

        graph = StateGraph(AgentState)
        graph.add_node(node="llm", action=self.call_openai)
        graph.add_node(node="action", action=self.take_action)
        graph.add_conditional_edges(
            source="llm", path=self.exists_action, path_map={True: "action", False: END}
        )
        graph.add_edge(start_key="action", end_key="llm")
        graph.set_entry_point(key="llm")

        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState) -> bool:
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState) -> dict:
        messages = state["messages"]

        if self.system:
            messages = [SystemMessage(content=self.system)] + messages

        message = self.model.invoke(messages)

        return {"messages": [message]}

    def take_action(self, state: AgentState) -> dict:
        tool_calls = state["messages"][-1].tool_calls
        results = []

        for t in tool_calls:
            print(f"Calling: {t}")

            result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result))
            )

        print("Back to the model!")

        return {"messages": results}

In [9]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

In [14]:
model = ChatOpenAI(model=MODEL, base_url=BASE_URL)

In [15]:
abot = Agent(model=model, tools=[tool], system=prompt)

In [16]:
messages = [HumanMessage(content="What is the weather like in San Francisco now?")]

result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'call_8726258812609962530'}
Back to the model!


In [19]:
for message in result["messages"]:
    print(message)

content='What is the weather like in San Francisco now?'
content='' additional_kwargs={'tool_calls': [{'id': 'call_8726258812609962530', 'function': {'arguments': '{"query":"San Francisco weather"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}]} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 214, 'total_tokens': 234}, 'model_name': 'glm-4', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-57683940-ce03-4821-b9a9-ca0f3295f84a-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'call_8726258812609962530'}] usage_metadata={'input_tokens': 214, 'output_tokens': 20, 'total_tokens': 234}
content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_e

In [20]:
print(result["messages"][-1].content)

According to the search results, the current weather condition in San Francisco is mist with a temperature of 9.0°C (48.2°F). It's night time and the wind speed is 3.6 km/h (2.2 mph) from the north. The visibility is 8.0 kilometers (4.0 miles) and the humidity is 100%. Please note that weather conditions may change, so it's always a good idea to check for updates before going outside.


In [21]:
messages = [
    HumanMessage(
        content="What is the weather like in San Francisco and Los Angeles now?"
    )
]

result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'call_8726254174045176622'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles weather'}, 'id': 'call_8726254002246480129'}
Back to the model!


In [22]:
print(result["messages"][-1].content)

The current weather in San Francisco is mist with a temperature of 9.0°C (48.2°F), while the current weather in Los Angeles is also mist with a temperature of 17.2°C (63.0°F).


In [23]:
query = "Who won the super bowl in 2024? What is the GDP of state where the winning team is located?"

messages = [HumanMessage(content=query)]

result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'super bowl 2024 winner'}, 'id': 'call_8721259058359544598'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'gdp of kansas city'}, 'id': 'call_8726252627856829320'}
Back to the model!


In [24]:
print(result["messages"][-1].content)

The Kansas City Chiefs won the Super Bowl in 2024. The GDP of Kansas City, MO-KS (MSA) was $64,342,000,000 in 2022.
